### **On-Policy Monte Carlo Control**

#### **1. On and Off Policy Learning**

*   **核心问题：** 如何在与环境交互的同时，学习一个最优策略？
*   **两种范式：**
    *   **On-Policy（同策略）：** 我们要评估和改进的策略 **`π`**，与**实际生成行为轨迹（样本）的策略是同一个**。
        *   **口号：** “我正在做的，就是我正在学的。”
        *   **特点：** 通常更简单、更直接。策略通常是“软性”的，即对所有状态下的所有动作都有非零的概率被选中，以保证持续的探索。
    *   **Off-Policy（异策略）：** 我们要评估和改进的策略 **`π`**（**目标策略**），与**实际生成行为轨迹的策略 **`μ`**（**行为策略**）** 是不同的。
        *   **口号：** “我看着别人怎么做，来学习我应该怎么做。”
        *   **特点：** 更强大、更通用。可以复用由任何策略产生的历史数据，也可以学习一个确定性的最优策略，同时用一个探索性的行为策略来收集数据。

**我们现在聚焦于 On-Policy 方法。**

---

#### **2. Generalised Policy Iteration (GPI)**

这是所有强化学习控制算法的宏观蓝图。

*   **核心思想：** 在两个过程之间交替进行，直至收敛到最优策略 `π*` 和最优价值函数 `q*`。
    1.  **策略评估：** 评估当前策略 `π` 有多好。（计算 `Q^π(s, a)`）
    2.  **策略改进：** 基于评估出的价值函数，让策略变得更好。（提升 `π`）

*   **可视化理解：**
    
    策略评估和策略改进相互竞争、相互推动。评估使价值函数与当前策略一致，而改进使策略在当前价值函数下变得贪婪。当策略不再改变时，它就稳定在了最优策略上。

---

#### **3. Generalised Policy Iteration with Action-Value Function**

为什么在模型未知（无模型）的情况下，我们必须使用**动作价值函数 Q(s, a)** 而不是状态价值函数 V(s)？

*   **关键原因：** 有了 `Q(s, a)`，策略改进可以在**不依赖环境模型** `p(s‘, r | s, a)` 的情况下进行。
*   **策略改进定理：** 对于任意状态 `s`，构造一个新策略 `π'`，使得：
    `π’(s) = argmax_{a} Q^π(s, a)`
    那么，对于所有状态 `s`，新策略 `π‘` 一定不差于旧策略 `π`，即 `V^π’(s) ≥ V^π(s)`。
*   **结论：** 只要我们能够估计出 `Q^π(s, a)`，我们就可以通过简单地“在每个状态选择Q值最大的动作”来改进策略。这就是GPI在无模型条件下的实现形式。

---

#### **4. $\epsilon$-Greedy Policy Improvement**

如果我们简单地在每个状态都采取贪婪动作（`π'(s) = argmax_a Q(s, a)`），会有什么问题？

*   **问题：** 会彻底失去探索。一旦某个非最优动作在某个状态下没有被选中，它的Q值可能永远得不到更新，我们也就永远无法知道它是不是实际上更好的选择。
*   **解决方案：** **$\epsilon$-Greedy 策略。**
    *   **定义：** 以很高的概率 `1 - $\epsilon$` 选择当前认为最好的动作（贪婪动作），以很小的概率 `$\epsilon$` **随机**从所有动作中选择一个。
    *   **数学表达：**
        `π(a | s) = { 1 - $\epsilon$ + $\epsilon$ / |A(s)|, if a = argmax_{a'} Q(s, a’) }`
        `π(a | s) = { $\epsilon$ / |A(s)|, otherwise }`
    *   **优点：**
        1.  它保证了持续的探索，因为所有动作都有被选中的可能。
        2.  它同样满足**策略改进定理**。可以证明，通过 $\epsilon$-greedy 策略改进得到的新策略，其价值函数 `V^{π'}(s)` 确实不低于旧策略 `V^π(s)`。这意味着我们仍然在朝着最优策略的方向前进。

---

#### **5. Monte Carlo Policy Improvement**

现在，我们将GPI和蒙特卡洛方法结合起来。

*   **目标：** 在不知道环境模型的情况下，通过与环境交互的完整经验轨迹来学习最优策略。
*   **方法：** 使用蒙特卡洛方法来估计动作价值函数 `Q(s, a)`。
*   **基本流程（On-Policy MC Control）：**
    1.  **初始化：** 随机初始化一个策略 `π` 和 Q表 `Q(s, a)`。
    2.  **循环（多个Episode）：**
        *   **策略评估（蒙特卡洛）：** 使用策略 `π` 生成一个完整的Episode（例如：`S0, A0, R1, S1, A1, R2, ..., ST`）。对于这个Episode中出现的每一个状态-动作对 `(s, a)`，计算其从该时刻起获得的实际回报 `G_t`。然后用这个 `G_t` 去更新 `Q(s, a)` 的估计（例如：增量平均 `Q(s, a) ← Q(s, a) + α (G_t - Q(s, a))`）。
        *   **策略改进：** 对于这个Episode中访问到的每一个状态 `s`，根据更新后的 `Q(s, ·)` 来改进策略 `π`，例如将其更新为关于 `Q(s, ·)` 的 $\epsilon$-greedy 策略。

---

#### **6. GLIE Monte Carlo Control**

基本的On-Policy MC Control能工作，但理论上不能保证收敛到最优策略 `π*`。为了确保收敛，我们需要满足 **GLIE** 条件。

*   **GLIE 定义：** Greedy in the Limit with Infinite Exploration（无限探索下的极限贪婪）
    1.  **无限探索：** 所有的状态-动作对 `(s, a)` 都会被访问无限次。
        `lim (N(s, a) → ∞)` for all s, a
    2.  **极限贪婪：** 策略最终会收敛到一个贪婪策略。
        `lim (π(a | s) → 1)` for `a = argmax_{a'} Q(s, a’)`, and to 0 otherwise.

*   **如何实现GLIE？**
    *   **GLIE MC Control 算法：**
        1.  使用 $\epsilon$-greedy 策略来采集Episode。
        2.  对于Episode中每个出现的状态-动作对 `(s, a)`：
            *   更新访问计数：`N(s, a) ← N(s, a) + 1`
            *   更新Q值：`Q(s, a) ← Q(s, a) + (1 / N(s, a)) * (G_t - Q(s, a))` （这里使用简单的平均，也可以用学习率 `α`）
        3.  **关键步骤：** 在策略改进阶段，**让探索概率 $\epsilon$ 逐渐衰减到0**。
            *   例如：`$\epsilon$ ← 1 / k`，其中 `k` 是Episode的计数。

*   **为什么GLIE有效？**
    *   在开始时，`$\epsilon$` 很大，保证了充分的探索，满足条件1。
    *   随着时间推移，`$\epsilon$` 逐渐减小到0，策略变得越来越贪婪，最终收敛到一个确定性的最优策略，满足条件2。
    *   在Q函数的估计上，因为每个 `(s, a)` 对被访问了无限次，根据大数定律，`Q(s, a)` 会收敛到真实的 `q_π(s, a)`。

### **总结**

你的提纲完美地展现了On-Policy MC Control的推导过程：

1.  **确立范式（On-Policy）** -> 
2.  **确立宏观框架（GPI）** -> 
3.  **确定实现GPI的工具（Action-Value Function Q）** -> 
4.  **解决贪婪探索矛盾（$\epsilon$-Greedy）** -> 
5.  **组合成基础算法（MC Policy Improvement）** -> 
6.  **为算法加上理论保证（GLIE MC Control）**



### On Policy TD Learning

### Off Policy Learning